In [1]:
import pandas as pd
import numpy as np
from ark_tweet_nlp import CMUTweetTagger
from tqdm import tqdm_notebook as tqdm
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
import pickle
import copy
import re

/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/qrg-researchlab/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [9]:
"""Loading the tagged data and the tagged """
data = pickle.load(open('./label_tag_data.p', 'rb'))
tagged_hashtags = pickle.load(open('./hashtags/tagged_hashtags_last.p', 'rb'))
! mkdir neighborhoods

In [4]:
"""Examining the type of patterns the verbs and adverbs appear in"""
lmtzr = WordNetLemmatizer()
actual_words_5 = []
actual_words_7 = []
actual_words_9 = []
unique_verbs = set()

for k in tqdm(range(data.shape[0])):
    local = data.tag_df.iloc[k]
    
    """Removing non interesting tags, tweet specific tags (emojis, existentials, numbers, URLs, the &, punctuation, unknown, @ mentions, determinants)"""
    local = local[local.tag.isin(['U','&',',','$','D','!','^','L','M','X','Y','E','G']).apply(lambda x: not x)]
    if local.shape[0]==0:
        actual_words_5.append([])
        actual_words_7.append([])
        actual_words_9.append([])
    else: 
        """Lemmatizing the words to remove the verb and adverb tokens to be considered"""
        local['word'] = local.T.apply(lambda x: lmtzr.lemmatize(x['word'].replace("'",''), x['tag'].lower()) if x['tag'] in ['V','R','A','N'] else x['word'].replace("'",''))

        """Extracting the verb and adverb patterns"""
        local_words_5 = []
        local_words_7 = []
        local_words_9 = []
        for i in range(local.shape[0]):
            w = local.word.iloc[i]
            if w.count('/')==1 and w.count('-')==0:
                w = w.split('/')
            if w.count('/')==0 and w.count('-')==1:
                w = w.replace('-','')
            if (local.tag.iloc[i] in ['V','R']) and ('#' not in w) and ('&' not in w) and not(any(str(k) in w for k in range(10))) and ('-' not in w) and ('/' not in w):
                neighborhood_5 = local[['word','tag']].iloc[max(0,i-2):min(local.shape[0],i+3)]
                neighborhood_5 = neighborhood_5[neighborhood_5.word.apply(lambda x: '#' not in x)]
                local_words_5.append(list(neighborhood_5.word))
                
                neighborhood_7 = local[['word','tag']].iloc[max(0,i-3):min(local.shape[0],i+4)]
                neighborhood_7 = neighborhood_7[neighborhood_7.word.apply(lambda x: '#' not in x)]
                local_words_7.append(list(neighborhood_7.word))
                
                neighborhood_9 = local[['word','tag']].iloc[max(0,i-4):min(local.shape[0],i+5)]
                neighborhood_9 = neighborhood_9[neighborhood_9.word.apply(lambda x: '#' not in x)]
                local_words_9.append(list(neighborhood_9.word))
                
            """If the word is a verb, add it to the bank of verbs"""
            if (local.tag.iloc[i]=='V') and ('#' not in w) and ('&' not in w) and not(any(str(k) in w for k in range(10))) and ('-' not in w) and ('/' not in w):
                if type(w)==list:
                    unique_verbs.update(set(w))
                else:
                    unique_verbs.add(w)

        actual_words_5.append(local_words_5)
        actual_words_7.append(local_words_7)
        actual_words_9.append(local_words_9)
        
    """Check on length of neighborhoods'list"""
    assert(len(actual_words_5)==k+1)

/home/qrg-researchlab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
"""Elementary cleaning step for normalization"""
for k in tqdm(range(len(actual_words_5))):
    actual_words_5[k] = [[w.replace("'",'').replace("-",'') for w in neighborhood] for neighborhood in actual_words_5[k]]
    actual_words_7[k] = [[w.replace("'",'').replace("-",'') for w in neighborhood] for neighborhood in actual_words_7[k]]
    actual_words_9[k] = [[w.replace("'",'').replace("-",'') for w in neighborhood] for neighborhood in actual_words_9[k]]

In [6]:
"""Saving the neighborhoods"""
pickle.dump(actual_words_5, open('./neighborhoods/actual_words_hashtag_free_5.p', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(actual_words_7, open('./neighborhoods/actual_words_hashtag_free_7.p', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(actual_words_9, open('./neighborhoods/actual_words_hashtag_free_9.p', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
"""Building the neighborhoods extracted from hashtag decomposition (obtained so far)"""
tuple_hashtags = {ht: tuple(list(pd.DataFrame(tagged_hashtags[ht], columns=['word','tag','score'])['word'].apply(lambda x: x.replace("'",'')))) for ht in tagged_hashtags.keys()}
hashtag_words = []

for k in tqdm(range(data.shape[0]), desc='Adding the decomposition as neighborhoods to the previous ones extracted'):
    local = data.tag_df.iloc[k]
    local_hashtag = []
    for i in range(local.shape[0]):
        if local.tag.iloc[i]=='#' or '#' in local.word.iloc[i]:
            ht = local.word.iloc[i]
            if ht in tuple_hashtags.keys():
                local_hashtag.append(list(tuple_hashtags[ht]))
    hashtag_words.append(local_hashtag)
    
for k in tqdm(range(len(hashtag_words)), desc='Removing leftover from tag file and removing underscores'):
    hashtag_words[k] = [[w.replace("'",'').replace("-",'') for w in neighborhood] for neighborhood in hashtag_words[k]]

In [12]:
pickle.dump(hashtag_words, open('./neighborhoods/hashtag_words.p','wb'), protocol=pickle.HIGHEST_PROTOCOL)